# mat2csv Notebook

Dieses Notebook lädt eine MATLAB `.mat`-Datei ein und exportiert die enthaltenen Arrays als CSV-Dateien.


In [1]:
import os
from scipy.io import loadmat
import pandas as pd
import numpy as np
import ultralytics
print(ultralytics.__version__)

C:\Users\Erik\.conda\envs\pytorch-gpu\python.exe
Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Erik\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
8.3.163


In [25]:
# mat2csv.ipynb

#``` python
# %% [markdown]
# # MAT → CSV Konverter
# 
# Dieses Notebook lädt eine `.mat`-Datei, entpackt alle Felder aus den MATLAB-Structs  
# und speichert sie in einer flat CSV ab.

# %% [code]
import scipy.io
import pandas as pd

# Pfad zur .mat-Datei
mat_path = r'C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\devkit\cars_train_annos.mat'

# laden
mat = scipy.io.loadmat(mat_path, struct_as_record=False, squeeze_me=True)

# Schau dir an, welche Keys es gibt:
print(mat.keys())
# z.B. dict_keys(['__header__', '__version__', '__globals__', 'class_names', 'synsets'])

# %% [code]
# Hilfsfunktion, um MATLAB-Structs in dicts zu überführen
def _todict(matobj):
    # 0) Abfangen von Nicht-Structs (Strings, Zahlen, Arrays ohne _fieldnames, ...)
    if not hasattr(matobj, "_fieldnames"):
        return matobj

    # ab hier ist matobj garantiert ein Struct
    dict_out = {}
    for fieldname in matobj._fieldnames:
        elem = getattr(matobj, fieldname)
        # falls verschachtelte Structs
        if hasattr(elem, "_fieldnames"):
            dict_out[fieldname] = _todict(elem)
        # falls Array von Structs
        elif isinstance(elem, (list, np.ndarray)) and elem.dtype == object:
            dict_out[fieldname] = [_todict(e) for e in elem]
        else:
            dict_out[fieldname] = elem
    return dict_out


# %% [code]
# Suche den Struct-Eintrag (z.B. 'labels' oder dein haupt-Struct)
# Häufig ist das der einzige mat[key] ohne Leading-Underscore
data_keys = [k for k in mat.keys() if not k.startswith('__')]
data_keys
# %% [code]
# Beispiel: Wir nehmen den ersten Key
key = data_keys[0]
struct = mat[key]

# Falls es ein Array von Structs ist:
if isinstance(struct, (list, np.ndarray)):
    records = [_todict(item) for item in struct]
else:
    records = [_todict(struct)]

# %% [code]
# In DataFrame überführen
df = pd.json_normalize(records)
df.head()

# %% [code]
# CSV schreiben
csv_path = r'C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\annotations_train.csv'
df.to_csv(csv_path, index=False)
print(f"CSV geschrieben nach `{csv_path}`")

dict_keys(['__header__', '__version__', '__globals__', 'annotations'])
CSV geschrieben nach `C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\annotations_train.csv`


In [21]:
import scipy.io
import pandas as pd
import numpy as np
from pathlib import Path

def mat_to_csv(mat_path: str, csv_path: str, key: str = 'class_names'):
    """
    Liest das MATLAB-File `mat_path`, extrahiert das Array unter `key` und
    schreibt es als CSV nach `csv_path` mit Spalten ['index','value'].
    """
    # 1) Laden
    mat = scipy.io.loadmat(mat_path, struct_as_record=False, squeeze_me=True)
    if key not in mat:
        raise KeyError(f"Key '{key}' nicht in {mat_path}. Verfügbare Keys: {list(mat.keys())}")
    arr = mat[key]
    # 2) DataFrame bauen
    df = pd.DataFrame({
        'index': np.arange(1, len(arr) + 1),
        'value': arr
    })
    # 3) Speichern
    out = Path(csv_path)
    out.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out, index=False)
    print(f"→ geschrieben: {out}")
    return df

In [22]:
mat_to_csv(r'C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\devkit\cars_meta.mat',r'C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\meta.csv')

→ geschrieben: C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\meta.csv


,index,value
0,1,AM General Hummer SUV 2000
1,2,Acura RL Sedan 2012
2,3,Acura TL Sedan 2012
3,4,Acura TL Type-S 2008
4,5,Acura TSX Sedan 2012
...,...,...
191,192,Volkswagen Beetle Hatchback 2012
192,193,Volvo C30 Hatchback 2012
193,194,Volvo 240 Sedan 1993
194,195,Volvo XC90 SUV 2007


In [27]:
import re
import scipy.io
import pandas as pd
import numpy as np
from pathlib import Path

# Liste der gängigen Fahrzeugtypen, wie sie in den Stanford Cars Bezeichnungen vorkommen
VEHICLE_TYPES = [
    'SUV', 'Sedan', 'Wagon', 'Coupe', 'Convertible',
    'Hatchback', 'Truck', 'Van', 'Minivan', 'Pickup',
]

def split_model_info(df: pd.DataFrame, col: str = 'value'):
    """
    Nimmt einen DataFrame mit einer Spalte `col`, die z.B. 'AM General Hummer SUV 2000' enthält,
    und extrahiert daraus drei Spalten:
      - model   (ohne Typ und Jahr)
      - vtype   (der gefundene Fahrzeugtyp oder NaN)
      - year    (vierstellige Jahreszahl oder NaN)
    """
    # unify spelling: Sport Car → Sports Car
    df[col] = df[col].str.replace(r'\bSport Car\b', 'Sports Car', regex=True)
    models, vtypes, years = [], [], []
    # compile Jahr-Pattern: am Ende 4 Ziffern
    year_pattern = re.compile(r'(\b19\d{2}\b|\b20\d{2}\b)$', re.IGNORECASE)

    # Fahrzeugtypen-Regex: word-boundary, join alternatives
    vtype_pattern = re.compile(
        r'\b(' + '|'.join(re.escape(t) for t in VEHICLE_TYPES) + r')\b',
        re.IGNORECASE
    )

    for entry in df[col]:
        # 1) Baujahr finden
        m_year = year_pattern.search(entry)
        year = m_year.group(0) if m_year else np.nan

        # entferne Jahr fürs weitere Parsen
        base = entry[:m_year.start()] if m_year else entry

        # 2) Typ finden
        m_type = vtype_pattern.search(base)
        vtype = m_type.group(0) if m_type else np.nan

        # Entferne Typ aus Base
        if m_type:
            # alles links und rechts vom Typ (ohne das Wort selbst) zusammenführen
            start, end = m_type.span()
            name = (base[:start] + base[end:]).strip()
        else:
            name = base.strip()

        models.append(name)
        vtypes.append(vtype)
        years.append(year)

    df['model'] = models
    df['vtype'] = vtypes
    df['year'] = years
    return df

def mat_to_csv_with_split(mat_path: str, csv_path: str, key: str = 'class_names'):
    """
    Lädt mat, extrahiert key-Array, wandelt in CSV um und splittet model/vtype/year.
    """
    # 1) laden & array holen
    mat = scipy.io.loadmat(mat_path, struct_as_record=False, squeeze_me=True)
    arr = mat.get(key)
    if arr is None:
        raise KeyError(f"Key '{key}' nicht in {mat_path}. Verfügbare: {list(mat.keys())}")

    # 2) Basistabelle
    df = pd.DataFrame({
        'index': np.arange(1, len(arr) + 1),
        'value': arr
    })

    # 3) Split in model / vtype / year
    df = split_model_info(df, col='value')

    # 4) CSV schreiben
    out = Path(csv_path)
    out.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out, index=False)
    print(f"→ geschrieben: {out}")
    return df


In [28]:
df = mat_to_csv_with_split(
    mat_path=r'C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\devkit\cars_meta.mat',
    csv_path=r'C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\meta_seperated.csv',
    key='class_names'
)

df.head()

→ geschrieben: C:\Users\Erik\Desktop\Uni\HLCV\data\stanford_cars\meta_seperated.csv


,index,value,model,vtype,year
0,1,AM General Hummer SUV 2000,AM General Hummer,SUV,2000
1,2,Acura RL Sedan 2012,Acura RL,Sedan,2012
2,3,Acura TL Sedan 2012,Acura TL,Sedan,2012
3,4,Acura TL Type-S 2008,Acura TL Type-S,NaN,2008
4,5,Acura TSX Sedan 2012,Acura TSX,Sedan,2012
